<a href="https://colab.research.google.com/github/Reptilefury/coursera-machine-learning/blob/main/SavingTheModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf


In [2]:
from tensorflow import keras

In [3]:
#We start by loading the dataset 
mnist = tf.keras.datasets.mnist.load_data()

In [4]:
#Split the dataset into training and testing
(x_train,y_train),(x_test,y_test) = mnist

In [5]:
#Normalize the data by turning it into floating point numbers in between 0 and 1
x_train , y_train = x_train/255.0 , y_train/255.0
x_test,y_test = x_test/255.0 , y_test/255.0

In [6]:
#We create a sequential model 
model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=x_train[0].shape),
      tf.keras.layers.Dense(300,activation="relu"),
      tf.keras.layers.Dense(100,activation="relu"),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(10)#10 neurons in the output layer because we have 10  classes to activate
])

In [7]:
predictions = model.predict(x_train[:1])

In [8]:
#Softmax function 
tf.nn.softmax(predictions)


<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[0.0364995 , 0.16082592, 0.07058035, 0.07894336, 0.08369876,
        0.14202495, 0.05524097, 0.08639129, 0.15432249, 0.13147241]],
      dtype=float32)>

In [9]:
#We create a loss function
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [10]:
loss_fn(y_test[3], predictions).numpy()

3.3104568

In [11]:
#We declare hyperparameters
model.compile(loss = loss_fn, optimizer='sgd',metrics=['accuracy'])

In [12]:
history = model.fit(x_train, y_train, epochs=5, callbacks=[])

Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0174 - accuracy: 0.0985
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 4.4480e-04 - accuracy: 0.0987
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 2.6553e-04 - accuracy: 0.0987
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 1.8246e-04 - accuracy: 0.0987
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 1.3634e-04 - accuracy: 0.0987


In [13]:
#Saving the model is as simple as 
model.save("MyModel.h5")

In [14]:
#Loading a saved model
model = tf.keras.models.load_model("MyModel.h5")

In [15]:
#We should implement early Stopping or checkpoints to save our model best parameters just incase our model starts to overfit or when there is no progress in the validation loss
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("MyModel.h5", save_best_only=True)

In [16]:
#The model.fit method has callback functions that we can pass a list of objects to call during , before and after training
#In this case we call the callback and pass in the checkpoint method to prevent our model from overfitting
#Another default callback called after every epoch is the validation and training loss to gauge our model perfomance after every training cycle or epoch
 
model.fit(x_train, y_train, epochs = 5,  callbacks=[checkpoint_cb])

Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 1.1277e-04 - accuracy: 0.0987
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 9.4030e-05 - accuracy: 0.0987
Epoch 3/5
1875/1875 [==============================] - 10s 6ms/step - loss: 8.5844e-05 - accuracy: 0.0987
Epoch 4/5
1875/1875 [==============================] - 14s 8ms/step - loss: 6.5430e-05 - accuracy: 0.0987
Epoch 5/5
1875/1875 [==============================] - 13s 7ms/step - loss: 6.4155e-05 - accuracy: 0.0987


In [ ]:
#Early stopping method will stop training when it measures no progress on the validation set
#Then it will roll back to the best model parameters
#The number of epochs can be set to a large value since the model will automatically stop training
EarlyStopping = tf.keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)
model.fit(x_train,y_train, epochs=100, callbacks=[checkpoint_cb,EarlyStopping])

#EarlyStopping = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

In [18]:
#We could create custom callbacks, to display the ratio between the validation loss and training loss to detect overfitting 

class PrintValCallBack(tf.keras.callbacks.Callback):
  def on_epoch_end(self,epoch,log):
    print("\nval:/train{:2f".format(logs["val_logs"]/logs["loss"]))

    #We could also call onTrain begin onTrain end onbatch begin on bacth end 
    #We could also use callbacks when Debugging

    #Using callbacks

Visualising using Tensorboard

In [32]:
import os 
root_logdir=os.path.join(os.curdir,"MyModel") 
def get_run_logdir():
  import time
  run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S") 
  return os.path.join(root_logdir,run_id)

In [33]:
run_logdir = get_run_logdir()

In [34]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)

In [35]:
history = model.fit(x_train, y_train, callbacks=[tensorboard_cb])

1875/1875 [==============================] - 7s 4ms/step - loss: 2.4003e-05 - accuracy: 0.0987


In [40]:
#Visualising using tensorboard
import os
root_rundir = os.path.join(os.curdir, "MyModel")
def get_run_logdir():
  import time
  run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
  return os.path.join(root_dir, run_id)

In [42]:
run_dir = get_run_logdir()

In [44]:
#Create a tensorboard callback
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_dir)


In [46]:
history = model.fit(x_train,y_train,epochs=5,callbacks=[tensorboard_cb])

Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 2.3467e-05 - accuracy: 0.0987
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 2.0554e-05 - accuracy: 0.0987
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 2.0148e-05 - accuracy: 0.0987
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 2.0338e-05 - accuracy: 0.0987
Epoch 5/5
1875/1875 [==============================] - 8s 4ms/step - loss: 2.0106e-05 - accuracy: 0.0987
